In [1]:
import json
from pathlib import Path

# 데이터 라벨링 함수
def label_data(image_folder_path):
    labeled_data = {'description': [], 'annotations': []}

    image_folder = Path(image_folder_path)
    for label_type in image_folder.glob("*"):
        for image_path in label_type.glob("*.jpg"):
            # 이미지 파일의 상대 경로 추출
            rel_image_path = image_path.relative_to(image_folder)

            # 이미지 파일의 이름에서 .jpg 확장자 제거
            image_name = image_path.stem

            # 라벨 파일 경로 생성
            annotation_path = next(label_type.glob(f"{image_name}*.json"), None)  
             
            
            if annotation_path is None:
                # 라벨 파일이 없는 경우 스킵
                print(f"Skipping image {image_path}, label file not found in {label_type}")
                continue
                
            try:
                with open(annotation_path, 'rb') as raw_data:
                    # 파일을 이진 모드로 열어서 텍스트로 디코딩할 때 변환 문제가 발생하면 replace로 대체
                    annotation_data = json.loads(raw_data.read().decode('utf-8', errors='replace'))

                # 여기서 라벨 데이터 형식에 맞게 필요한 정보 추출
                disease_label = annotation_data['annotations']['disease']
                bounding_box = annotation_data['annotations']['points']
                original_image = annotation_data['description']['image']

                labeled_data['description'].append({'image': original_image})
                labeled_data['annotations'].append({'image': original_image, 'points': bounding_box, 'disease': disease_label})

            except json.JSONDecodeError as e:
                # JSON 디코딩 오류가 있는 경우 스킵
                print(f"Skipping image {image_path}, JSON decoding error in label file: {annotation_path}")
                print(f"Error details: {e}")
                continue

    return labeled_data

# 라벨링 함수 호출 및 JSON 파일 저장
labeled_data = label_data('./data/test/')  # 이미지 데이터 폴더에 맞게 수정
output_json_path = './data/test_labeled_data.json'
with open(output_json_path, 'w') as f:
    json.dump(labeled_data, f, ensure_ascii=False)

print(f"Labeled data saved to {output_json_path}")


Labeled data saved to ./data/test_labeled_data.json
